In [ ]:
import nltk as nl
import pandas as pd
import numpy as np
from PyDictionary import PyDictionary
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVR, SVR
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
import operator
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import KFold
from sklearn.ensemble import VotingClassifier
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, accuracy_score, make_scorer

import xgboost
from nltk.sentiment import SentimentAnalyzer # sentiment analysis
from nltk.tokenize import RegexpTokenizer #tokenization
from nltk.tokenize import TweetTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer # sentiment analysis VADER
from nltk.tokenize import sent_tokenize #sentence tokenizer
from sklearn.naive_bayes import GaussianNB

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [ ]:
def normalize(pd):
    featNorm = pd.copy()
    for k in range(len(pd.columns)):
        m = np.mean(pd.iloc[:,k])
        s = np.std(pd.iloc[:,k])
        featNorm.iloc[:,k] = (pd.iloc[:,k]-m)/s
    featNorm = featNorm.iloc[:,1:k]
    return(featNorm)

In [ ]:
def getAccuracyFunction(classifier, featuresel, numFeat,df):
    accuracy = getAccuracy(classifier, featuresel, numFeat, df, df.iloc[:,range(len(df.columns)-1)])
    return(accuracy)

In [ ]:
classifiers = ["Log","SVM","RandomForest","XgBoost","NaiveBayes"]
nrFeats = [10,50,100,150,250,300,350,400,450, 500,550,600]
resultsposPatterns=pd.DataFrame(index = nrFeats)
for classif in classifiers:
    print(classif)
    accuracies=[]
    for nfeat in nrFeats:
        [acc1,acc2]=getAccuracyFunction(classif,"f_classif",nfeat,allFClass)
        accuracies.append(acc1)
        print(nfeat)
    resultsposPatterns[classif]=accuracies


# MSE and ACCURACY


In [ ]:
posPatterns = pd.read_csv("posPatterns_ALL_SVM.csv")
unigrams = pd.read_csv("unigrams_select250_Log.csv")
mixed = pd.read_csv("mixtureDF_AllFeatures_SVM.csv") 
allfeat = pd.read_csv("features_barbara_all.csv")

posPatternClass = posPatterns.iloc[:,1:len(posPatterns.columns)]
unigramsClass = unigrams.iloc[:,1:len(unigrams.columns)]
mixedClass = mixed.iloc[:,1:len(mixed.columns)]
allfeatClass = allfeat.iloc[:,1:len(allfeat)]

posPatternM = posPatterns.iloc[:,1:len(posPatterns.columns)-1]
unigramsM = unigrams.iloc[:,1:len(unigrams.columns)-1]
mixedM = mixed.iloc[:,1:len(mixed.columns)-1]
allfeatM = allfeat.iloc[:,1:len(allfeat.columns)-1]

posPatternM = pd.merge(posPatternM, Labels[["id","truthMean"]], on = "id")
unigramsM = pd.merge(unigramsM, Labels[["id","truthMean"]], on = "id")
mixedM = pd.merge(mixedM, Labels[["id","truthMean"]], on = "id")
allfeatM = pd.merge(allfeatM, Labels[["id","truthMean"]], on = "id")

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [ ]:
mixedClass
# content = mixedClass[["id"]].copy()
sentiment = mixedClass.iloc[:,0:4]
informality = mixedClass[["id"]].copy()
sentiment
informality[["inform_CLScore", "inform_RIX",  "inform_LIX", "inform_fmeas" ]] = mixedClass[["inform_CLScore", "inform_RIX",  "inform_LIX", "inform_fmeas" ]]
content = mixedClass.drop(["sentimentPostText", "sentimentTargetText", "difsentimentPostTarget","inform_CLScore", "inform_RIX",  "inform_LIX", "inform_fmeas"], axis = 1 )
content = content.drop(["truthClass"], axis = 1)
content

In [ ]:
contentM = pd.merge(content, Labels[["id","truthMean"]], on = "id")
informalityM = pd.merge(informality, Labels[["id","truthMean"]], on = "id")
sentimentM = pd.merge(sentiment, Labels[["id","truthMean"]], on = "id")

contentClass = pd.merge(content, Labels[["id","truthClass"]], on = "id")
informalityClass = pd.merge(informality, Labels[["id","truthClass"]], on = "id")
sentimentClass = pd.merge(sentiment, Labels[["id","truthClass"]], on = "id")



In [ ]:
def getAccuracyVal(classifier, featuresel, numFeat, allfeatures, featurelist = finalFeatures):
    #Choose Classifier
    if(classifier == "SVM"):
        clf = SVC()
    elif(classifier == "Log"):
        clf = LogisticRegressionCV()
    elif(classifier == "RandomForest"):
        clf = RandomForestClassifier(criterion = 'entropy')
    elif(classifier == "XgBoost"):
        clf = GradientBoostingClassifier()
    elif(classifier == "NaiveBayes"):
        clf = GaussianNB()
    
    

        
    # select features we want
    x = [col for col in allfeatures if col in featurelist]
    x = allfeatures[x]

    
    featNorm = normalize(x)

    y = allfeatures.iloc[:,-1]
    
    if(featuresel== "chi"):
        X_new = SelectKBest(chi2, k=numFeat).fit_transform(featNorm, y)
    elif(featuresel == "f_classif"):
        X_new = SelectKBest(f_classif, k=numFeat).fit_transform(featNorm, y)
    elif(featuresel == "mutual_info"):
        X_new = SelectKBest(mutual_info_classif, k=numFeat).fit_transform(featNorm, y)
    else:
        X_new = featNorm
    #Classifier result
    X_train = X_new[0:2459,:]
    y_train = y[0:2459]
    X_test = X_new[2459:,:] 
    y_test = y[2459:]
    clf.fit(X_train, y_train)
    accuracy = clf.score(X_test, y_test)

    return(accuracy)
    

In [ ]:


def computeMSE(finalFeaturesMSE, finalFeatures, cross_val, featuresel, numFeat, regressor, param):
    
    ##set up cross validation
    cv_idx = KFold(n=len(varpd), n_folds=cross_val, shuffle=True, random_state=1)
    
    featNormMSE = normalize(finalFeaturesMSE)
    
    #only works with nd numpy arrays
    y = finalFeaturesMSE.truthMean.values
    X = featNormMSE.as_matrix(columns = featNormMSE.columns[1:len(featNormMSE.columns)-1])
    yhard = finalFeatures.truthClass.values
    
    #feature selection
    if(featuresel== "chi"):
        X_new = SelectKBest(chi2, k=numFeat).fit_transform(X, y)
    elif(featuresel == "f_classif"):
        X_new = SelectKBest(f_classif, k=numFeat).fit_transform(X, y)
    elif(featuresel == "mutual_info"):
        X_new = SelectKBest(mutual_info_classif, k=numFeat).fit_transform(X, y)
    else:
        X_new = X

    y_out = pd.DataFrame()
    scores = []
    acc= []
    count = 0
    
    #cross validation and select classifier/regressor
    for train_idx, val_idx in cv_idx:
            if(regressor == "LinearSVR"):
                clr = LinearSVR(C=param, loss='squared_epsilon_insensitive', dual=False, random_state=1)
                clr.fit(X_new[train_idx], y[train_idx])
                y_pred = clr.predict(X_new[val_idx])           
                lab_pred = y_pred.copy() # i added this to work with regression
                lab_pred=np.where(lab_pred >= 0.5, 1,-1)
                score = accuracy_score(lab_pred,yhard[val_idx])
                acc.append(score)

            elif(regressor == "Log"):
                clr = LogisticRegression()
                clr.fit(X_new[train_idx], yhard[train_idx])
                y_pred =clr.predict_proba(X_new[val_idx])[:,1]
            elif(regressor == "XgBoost"):
                clr = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)
                clr.fit(X_new[train_idx], y[train_idx])
                y_pred = clr.predict(X_new[val_idx])
                lab_pred = y_pred.copy() # i added this to work with regression
                lab_pred=np.where(lab_pred >= 0.5, 1,-1)
                score = accuracy_score(lab_pred,yhard[val_idx])
                acc.append(score)

            if(regressor == "RFreg"):
                clr = RandomForestRegressor(max_depth=param, random_state=0) 
                
                clr.fit(X_new[train_idx], y[train_idx]) 
                
                y_pred = clr.predict(X_new[val_idx])
                lab_pred = y_pred.copy() # i added this to work with regression
                lab_pred=np.where(lab_pred >= 0.5, 1,-1)
                score = accuracy_score(lab_pred,yhard[val_idx])
                acc.append(score)
            elif(regressor == "SVC"):
                clr = SVC(probability = True)
                clr.fit(X_new[train_idx], yhard[train_idx])
                y_pred =clr.predict_proba(X_new[val_idx])[:,1]
            elif(regressor == "Xg"):
                clr = GradientBoostingClassifier()
                clr.fit(X_new[train_idx], yhard[train_idx])
                y_pred =clr.predict_proba(X_new[val_idx])[:,1]
            elif(regressor == "RF"):
                clr = RandomForestClassifier(criterion = 'entropy')
                clr.fit(X_new[train_idx], yhard[train_idx])
                y_pred =clr.predict_proba(X_new[val_idx])[:,1]
            elif(regressor == "NB"):
                clr = GaussianNB()
                clr.fit(X_new[train_idx], yhard[train_idx])
                y_pred =clr.predict_proba(X_new[val_idx])[:,1]

    


            rmse = mean_squared_error(y[val_idx], y_pred)
            scores.append(rmse)

    m = np.mean(scores)
    s = np.std(scores)
    print("MSE: %0.3f (+/- %0.2f)" % (m, s))
    
    mA = np.mean(acc)
    sA = np.std(acc)
    print("Accuracy: %0.3f (+/- %0.2f)" % (mA, sA))
    
    return(m , mA)

In [ ]:
def computeEnsembleClassifier(posPatternClass, unigramsClass,  mixedClass, finalFeatures, finalFeaturesMSE, cross_val,c, cls):
    
    ##set up cross validation
    cv_idx = KFold(n=len(varpd), n_folds=cross_val, shuffle=True, random_state=1)
    
    originalclass = []
    predictedclass = []
    
    yclass = finalFeatures.iloc[:,-1]
    posPatternMMSE = normalize(posPatternClass)
    unigramsMMSE = normalize(unigramsClass)
    mixedMMSE = normalize(mixedClass)
    finalFeaturesNormMSE = normalize(finalFeatures)
    
    #only works with nd numpy arrays
    
    ycont = finalFeaturesMSE.truthMean.values
    yp = posPatternClass.truthClass.values
    yu = unigramsClass.truthClass.values
    ym = mixedClass.truthClass.values
    yf = finalFeatures.truthClass.values
    
    Xp = posPatternMMSE.as_matrix(columns = posPatternMMSE.columns)
    Xu = unigramsMMSE.as_matrix(columns = unigramsMMSE.columns)
    Xm = mixedMMSE.as_matrix(columns = mixedMMSE.columns)
    Xf = finalFeaturesNormMSE.as_matrix(columns = finalFeaturesNormMSE.columns)
    

    Xp_new = SelectKBest(f_classif, k="all").fit_transform(Xp, yp)
    Xu_new = SelectKBest(f_classif, k=250).fit_transform(Xu, yu)
    Xm_new = SelectKBest(f_classif, k="all").fit_transform(Xm, ym)
    Xf_new = SelectKBest(f_classif, k="all").fit_transform(Xf, yf)
    
    
    y_out = pd.DataFrame()

    accM = []
    accS = []

    count = 0
    for train_idx, val_idx in cv_idx:
           
            # select best classifier for each group
            clrp = SVC(probability = True)
            clru = LogisticRegressionCV()
            clrm = SVC(probability = True)
            clrf = LogisticRegressionCV()
                           
            clrp.fit(Xp_new[train_idx], yp[train_idx])
            clru.fit(Xu_new[train_idx], yu[train_idx])
            clrm.fit(Xm_new[train_idx], ym[train_idx])
            clrf.fit(Xf_new[train_idx], yf[train_idx])
            
            #calculate posterior probabilities
            yp_pred = clrp.predict_proba(Xp_new[val_idx])[:,1]
            yp_pred_train = clrp.predict_proba(Xp_new[train_idx])[:,1]

            
            
            yu_pred = clru.predict_proba(Xu_new[val_idx])[:,1]
            yu_pred_train = clru.predict_proba(Xu_new[train_idx])[:,1]
            
            ym_pred = clrm.predict_proba(Xm_new[val_idx])[:,1]
            ym_pred_train = clrm.predict_proba(Xm_new[train_idx])[:,1]
           
            
            yf_pred = clrf.predict_proba(Xf_new[val_idx])[:,1]
            yf_pred_train = clrf.predict_proba(Xf_new[train_idx])[:,1]
            

            
            d1= {'p':yp_pred.tolist(), 'u':yu_pred.tolist(), 'm':ym_pred.tolist(), 'f':yf_pred.tolist(), 'y': yclass[val_idx].tolist(), 'ycont': ycont[val_idx].tolist()}
            d2= {'p':yp_pred_train.tolist(), 'u':yu_pred_train.tolist(), 'm':ym_pred_train.tolist(), 'f':yf_pred_train.tolist(), 'y': yclass[train_idx].tolist(), 'ycont': ycont[train_idx].tolist()}
            df1 = pd.DataFrame(d1)
            df2 = pd.DataFrame(d2)
            
            d= pd.concat([df1, df2])
            
            X_train, y_train, ycont_train = df2[["p", "u", "m", "f"]], df2["y"], df2["ycont"]
            #test = d[~msk]
            X_test, y_test, ycont_test = df1[["p", "u", "m", "f"]], df1["y"], df1["ycont"]
            
            #feed into gressor
            if(cls == "RF"):
                clf = RandomForestRegressor(max_depth=c, random_state=0) #i used random forest
                #             prob = cl.predict_proba
                clf.fit(X_train, ycont_train) 
                #score = clf.score(X_test, ycont_test) 
                prob = clf.predict(X_test)
            
            
            elif(cls == "SVC"):
                clf =  LinearSVR(C=c, loss='squared_epsilon_insensitive', dual=False, random_state=1)
                #             prob = cl.predict_proba
                clf.fit(X_train, ycont_train) 
                #score = clf.score(X_test, ycont_test) 
                prob = clf.predict(X_test)
            
            
            elif(cls == "Log"):
                clf = LogisticRegressionCV()
                clf.fit(X_train, y_train)
                prob = clf.predict_proba(X_test)[:,1]
            elif(cls == "Xg"):
                clf = GradientBoostingClassifier()
                clf.fit(X_train, y_train)
                prob = clf.predict_proba(X_test)[:,1] 
            elif(cls == "XgBoost"):
                clf =  xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)
                #             prob = cl.predict_proba
                clf.fit(X_train, ycont_train) 
                
                prob = clf.predict(X_test)

            # calculate accuracy and MSE
            lab_pred = prob.copy()
            lab_pred=np.where(lab_pred >= 0.5, 1,-1)
            score = accuracy_score(lab_pred,y_test)
            rmse = mean_squared_error(ycont_test, prob)

            
            accM.append(score)
            accS.append(rmse)
            
            originalclass.extend(y_test)
            predictedclass.extend(lab_pred)


    mS = np.mean(accS)
    sS = np.std(accS)
    print("MSE: %0.3f (+/- %0.2f)" % (mS, sS))

#     return(d)
    mA = np.mean(accM)
    sA = np.std(accM)
    print("Accuracy: %0.3f (+/- %0.2f)" % (mA, sA))
    
    print(classification_report(originalclass, predictedclass)) 
    
